# Feature Engineering

For some definitions, handling outliers and missing values, scaling, and encoding may be considered feature engineering. Here we'll draw a distinction between data preparation, data preprocessing, and feature engineering.

- **data preparation**: the basic data cleaning necessary to get our data ready for exploration/analysis, e.g. correcting data types, fixing typos
- **data preprocessing**: further data transformation done for the sake of modeling, as oppsoed to exploration/analysis, e.g. scaling, imputing, encoding
- **feature engineering**: adding, combining, or removing features; usually with the help of domain knowledge

Feature engineering can happen as part of data exploration or modeling, and engineered featured are also commonly explored.

Some examples of feature engineering by this definition:

- domain-based conversion (example: farenheit to celsius, BMI calculation, log transformation)
- domain based cutoffs (example: age >= 18 = is_adult; also dates)

- add / subtract (example: zillow dataset: beds + baths = room_count; total_sqft - 200 * bedrooms - 40 * bathrooms = living_area)
- combine as booleans as a count (example: telco_churn: streaming + backups + ...  = service_count)
- multiply / divide (example: tips dataset: total_bill / size = price_per_person)
- ratios (example: tips dataset: tip / total_bill = tip percentage)

Simplify!

- categorical with many unique values to top 3 + "Other"
- categorical to boolean: pool count -> has pool
- continous -> categorical via binning (aka quantization or discretization) (example: income -> high, medium, low earner)

In this lesson we'll cover some *automated* **feature selection** methods, that is, methods for determining which features are the most important.

- SelectKBest
- Recursive Feature Elimination
- Sequential Feature Selection

## Setup

In [2]:
import numpy as np
import pandas as pd
import wrangle

from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

In [3]:
df = wrangle.wrangle_grades()

In [4]:
# train_validate, test = train_test_split(df)
# train, validate = train_test_split(train_validate)

train, validate, test = wrangle.split_continuous(df)

train -> (56, 5)
validate -> (25, 5)
test -> (21, 5)


In [5]:
train.head()

,student_id,exam1,exam2,exam3,final_grade
51,52,70,75,78,72
24,25,57,65,75,65
57,58,79,70,85,81
92,93,98,93,96,95
81,82,83,80,86,85


In [6]:
train = train.drop(columns='student_id')

## Select K Best

- looks at each feature in isolation against the target based on correlation
- fastest of all approaches covered in this lesson
- doesn't consider feature interactions
- After fitting: `.scores_`, `.pvalues_`, `.get_support()`, and `.transform`

In [9]:
# X_trian, y_train 
X_train = train.drop(columns='final_grade')
y_train = train['final_grade']

In [46]:
# make the thing
kbest = SelectKBest(f_regression, k=1)
# fit the thing
_ = kbest.fit(X_train, y_train)

In [47]:
# statistical f-value:
kbest.scores_
#p value: 
kbest.pvalues_

array([1.00651044e-43, 2.70194813e-25, 6.39985139e-29])

In [48]:
# get-support() will output a boolean mask to tell me which features were selected
# we can apply this mask to the columns in our original dataframe


In [49]:
kbest.get_support()

array([ True, False, False])

In [50]:
# I can use the boolean mask of get_support on columns
X_train.columns[kbest.get_support()]

Index(['exam1'], dtype='object')

In [24]:
# I can use it as a 1-axis mapping with iloc:
X_train.iloc[:,kbest.get_support()].head()

,exam1,exam3
51,70,78
24,57,75
57,79,85
92,98,96
81,83,86


In [ ]:
# kbest transform will convert our information to the selected
# feature subspace
# ****buuuuuut, its just a numpy array


In [25]:
kbest.transform(X_train)

array([[ 70,  78],
       [ 57,  75],
       [ 79,  85],
       [ 98,  96],
       [ 83,  86],
       [ 83,  86],
       [ 58,  70],
       [ 92,  94],
       [ 58,  70],
       [ 85,  87],
       [ 70,  78],
       [ 73,  75],
       [ 58,  70],
       [ 98,  96],
       [ 62,  79],
       [ 62,  79],
       [ 85,  87],
       [ 58,  70],
       [ 62,  79],
       [ 70,  78],
       [ 79,  85],
       [ 85,  87],
       [ 73,  75],
       [ 57,  75],
       [ 73,  75],
       [ 57,  75],
       [ 57,  75],
       [ 83,  86],
       [ 85,  87],
       [ 73,  75],
       [100,  95],
       [ 92,  94],
       [ 92,  94],
       [ 83,  86],
       [ 93,  96],
       [ 92,  94],
       [ 73,  75],
       [ 83,  86],
       [ 92,  94],
       [ 93,  96],
       [ 92,  94],
       [ 93,  96],
       [ 79,  85],
       [ 85,  87],
       [ 57,  75],
       [ 83,  86],
       [ 70,  78],
       [ 73,  75],
       [100,  95],
       [ 62,  79],
       [ 70,  78],
       [ 58,  70],
       [ 98,

## RFE

- Recursive Feature Elimination
- Progressively eliminate features based on importance to the model
- Requires a model with either a `.coef_` or `.feature_importances_` property
- After fitting: `.ranking_`, `.get_support()`, and `.transform()`

In [26]:
from sklearn.linear_model import LinearRegression

In [51]:
# etablish a model for RFE to use: (make a model thing)
# linear model works the same as in any other context here, so do all the preprocessing
# that you would normally do to those features before you get to this point!
model = LinearRegression()

In [52]:
# make an RFE thing
rfe = RFE(model, n_features_to_select=1)

In [53]:
# fit the RFE:
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=1)

In [54]:
rfe.ranking_

array([1, 3, 2])

In [55]:
X_train.columns

Index(['exam1', 'exam2', 'exam3'], dtype='object')

New discovery: scientists have now found a new species of support called .support_

In [56]:
# get_support() still works to get your boolean mask
rfe.get_support()

array([ True, False, False])

In [35]:
# so does .support_ (but only for RFE, not kbest)
rfe.support_

array([ True, False,  True])

In [32]:
pd.DataFrame({
    'rfe_ranking': rfe.ranking_}, index=X_train.columns)

,rfe_ranking
exam1,1
exam2,2
exam3,1


In [40]:
pd.DataFrame(rfe.transform(X_train),
             columns=X_train.columns[rfe.get_support()],
             index=X_train.index).head()

,exam1,exam3
51,70,78
24,57,75
57,79,85
92,98,96
81,83,86


## Sequential Feature Selector

- progressively adds features based on cross validated model performance
- forwards: start with 0, add the best additional feature until you have the desired number
- backwards: start with all features, remove the worst performing until you have the desired number
- After fitting: `.support_`, `.transform`

In [42]:
# implement sequential feature selector:
# make a model:
model = LinearRegression()
# make a sequential feature selection object:
sfs = SequentialFeatureSelector(model, n_features_to_select=2)
# fit the feature selector:
sfs.fit(X_train, y_train)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=2)

In [44]:
sfs.get_support()

array([ True, False,  True])

In [45]:
X_train.columns[sfs.get_support()]

Index(['exam1', 'exam3'], dtype='object')

In [43]:
sfs.transform(X_train)

array([[ 70,  78],
       [ 57,  75],
       [ 79,  85],
       [ 98,  96],
       [ 83,  86],
       [ 83,  86],
       [ 58,  70],
       [ 92,  94],
       [ 58,  70],
       [ 85,  87],
       [ 70,  78],
       [ 73,  75],
       [ 58,  70],
       [ 98,  96],
       [ 62,  79],
       [ 62,  79],
       [ 85,  87],
       [ 58,  70],
       [ 62,  79],
       [ 70,  78],
       [ 79,  85],
       [ 85,  87],
       [ 73,  75],
       [ 57,  75],
       [ 73,  75],
       [ 57,  75],
       [ 57,  75],
       [ 83,  86],
       [ 85,  87],
       [ 73,  75],
       [100,  95],
       [ 92,  94],
       [ 92,  94],
       [ 83,  86],
       [ 93,  96],
       [ 92,  94],
       [ 73,  75],
       [ 83,  86],
       [ 92,  94],
       [ 93,  96],
       [ 92,  94],
       [ 93,  96],
       [ 79,  85],
       [ 85,  87],
       [ 57,  75],
       [ 83,  86],
       [ 70,  78],
       [ 73,  75],
       [100,  95],
       [ 62,  79],
       [ 70,  78],
       [ 58,  70],
       [ 98,

## Conclusion

- Simpler models handle change + variability better
- Use RFE to narrow down your features and find the best ones, if your dataset is large (> 1GB; `df.info()`) use select k best instead
- Remember: feature engineering is much more than feature selection!